# Importing Libraries

In [1]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator

In [2]:
print(tf.__version__)

2.4.0


# Part 1:Data Preprocessing

## Preprocessing the training set

In [3]:
# rescale - Feature scaling - 255 is max value of scaling(0-255).
# shear_range, zoom_range, horizontal_flip - to prevent overfitting
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

train_set = train_datagen.flow_from_directory(
        'dataset/training_set',
        target_size=(64, 64), # Final size of Image
        batch_size=32,        # 32 training records(row) in 1 batch
        class_mode='binary') # Binary or categorial

Found 8000 images belonging to 2 classes.


## Preprocessing testset

In [4]:
#We shouldn't apply rest of t parameter as in training. Same as ML - fit_transform in training set 
#but only transform the test set
test_datagen = ImageDataGenerator(rescale=1./255)
test_set = test_datagen.flow_from_directory(
        'dataset/test_set',
        target_size=(64, 64),
        batch_size=32,
        class_mode='binary')

Found 2000 images belonging to 2 classes.


# Part 2:Building CNN

## Initialising CNN

In [6]:
cnn = tf.keras.models.Sequential()

## Step 1 - Convolution

In [7]:
# filters-32 different filter which detects vertical, horizontal edges, flip, blur etc
# kernel_size - 3x3 Matrix for Feature detection. input_shape- Size of image as mentioned in preprocessing(64,64)& 3- RGB(since its colored image) 
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64,64,3]))

## Step 2 - Pooling

In [8]:
# pool_size - 2x2 pixel Sizing and strides move by 2 pixel. Refer TensorFlow-Udemy in /home/vinoth/Learning/Python
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

## Adding a second convolution layer

In [9]:
# kernel_size - 3x3 Matrix for Feature detection. input_shape is required only when connecting 1st layer to input
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
# pool_size - 2x2 pixel Sizing and strides move by 2 pixel. Refer TensorFlow-Udemy in /home/vinoth/Learning/Python
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

## Step 3 - Flattening

In [10]:
cnn.add(tf.keras.layers.Flatten())

## Step 4 - Full Connection

In [11]:
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

## Step 5 - Output Layer

In [12]:
# units=1&activation='sigmoid' since its binary output
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Part 3 - Training CNN

## Compiling CNN

Updating weight of each neuron in hidden layer to reduce the loss over the epoch. Signal from Feature or dependent variable is forward propagated to hidden layers and then to output layer. Predicted result is compared to actual result in Energy output. This incur the loss which is exactly squared difference between predicted vs actual energy output. This predictions comes in batches which will sum squared difference between predicted vs actual energy output, incurs loss. So, backward propogation is applied to neural network. Adam Stochastic gradient descent is applied with optimiser to reduce the loss.

In [13]:
cnn.compile(optimizer='adam', loss= 'binary_crossentropy', metrics= ['accuracy'])

## Training CNN on training set and evaluating the test set

Finally, let’s make this concrete with a small example.

Assume you have a dataset with 200 samples (rows of data) and you choose a batch size of 5 and 1,000 epochs.

This means that the dataset will be divided into 40 batches, each with five samples. The model weights will be updated after each batch of five samples.

This also means that one epoch will involve 40 batches or 40 updates to the model.

With 1,000 epochs, the model will be exposed to or pass through the whole dataset 1,000 times. That is a total of 40,000 batches during the entire training process.

https://machinelearningmastery.com/difference-between-a-batch-and-an-epoch/


In [14]:
cnn.fit(x=train_set, validation_data=test_set, epochs=25)

Epoch 1/25
250/250 [==============================] - 244s 963ms/step - loss: 0.6926 - accuracy: 0.5279 - val_loss: 0.6030 - val_accuracy: 0.6880
Epoch 2/25
250/250 [==============================] - 228s 911ms/step - loss: 0.6140 - accuracy: 0.6648 - val_loss: 0.5899 - val_accuracy: 0.7140
Epoch 3/25
250/250 [==============================] - 217s 868ms/step - loss: 0.5834 - accuracy: 0.6964 - val_loss: 0.5597 - val_accuracy: 0.7230
Epoch 4/25
250/250 [==============================] - 145s 579ms/step - loss: 0.5540 - accuracy: 0.7189 - val_loss: 0.5223 - val_accuracy: 0.7460
Epoch 5/25
250/250 [==============================] - 142s 567ms/step - loss: 0.5131 - accuracy: 0.7428 - val_loss: 0.4948 - val_accuracy: 0.7645
Epoch 6/25
250/250 [==============================] - 115s 460ms/step - loss: 0.5016 - accuracy: 0.7516 - val_loss: 0.5122 - val_accuracy: 0.7585
Epoch 7/25
250/250 [==============================] - 126s 506ms/step - loss: 0.4627 - accuracy: 0.7791 - val_loss: 0.4834 -

# Part 4: Making single prediction

In [39]:
import numpy as np
from keras.preprocessing import image
test_image = image.load_img('dataset/single_prediction/cat_or_dog_2.jpg', target_size = (64, 64))
test_image = image.img_to_array(test_image)
#print(test_image)
test_image = np.expand_dims(test_image, axis = 0)
#print(test_image)
result = cnn.predict(test_image)
train_set.class_indices
print('result : ', result)
if result[0][0] == 1:
  prediction = 'dog'
else:
  prediction = 'cat'

print('Prediction : ', prediction)

result :  [[1.]]
Prediction :  dog


In [40]:
gfg = np.array([[1, 2], [7, 8]]) 
print(gfg)
print(gfg.shape) 
  
gfg = np.expand_dims(gfg, axis = 0) 
print(gfg)
print(gfg.shape)

[[1 2]
 [7 8]]
(2, 2)
[[[1 2]
  [7 8]]]
(1, 2, 2)


In [46]:
import numpy as np
from keras.preprocessing import image
test_image = image.load_img('dataset/single_prediction/cat_or_dog_4.jpg', target_size = (64, 64))
test_image = image.img_to_array(test_image)
#print(test_image)
test_image = np.expand_dims(test_image, axis = 0)
#print(test_image)
result = cnn.predict(test_image)
train_set.class_indices
print('result : ', result)
if result[0][0] == 1:
  prediction = 'dog'
else:
  prediction = 'cat'

print('Prediction : ', prediction)

result :  [[1.]]
Prediction :  dog
